In [1]:
import requests
import geopandas as gpd
import json

# API endpoint URL
# url is for S0103 data from the Garfield County, CO 2022 American Community Survey
# This url is found on the census bureau website under "more tools" and "api"
garfieldUrl = "https://api.census.gov/data/2022/acs/acs5/subject?get=group(S0103)&ucgid=0500000US08045"

try:
    # Fetch data from the API
    response = requests.get(garfieldUrl)

    # Check if request was successful (status code 200)
    if response.status_code == 200:
        # Parse JSON data
        data = response.json()

        # Convert to GeoDataFrame (assuming your data format allows GeoDataFrame creation)
        # Example: Convert JSON data to a DataFrame and then to a GeoDataFrame
        # This depends on how your JSON response is structured
        garfield = gpd.GeoDataFrame(data[1:], columns=data[0])  # Assuming first row is headers

        # Print first few rows
        print(garfield.head())

    else:
        print(f"Error fetching data. Status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")


           GEO_ID                       NAME S0103_C01_001E S0103_C01_001EA  \
0  0500000US08045  Garfield County, Colorado          61683            None   

  S0103_C01_001M S0103_C01_001MA S0103_C01_002E S0103_C01_002EA  \
0     -555555555           *****           51.4            None   

  S0103_C01_002M S0103_C01_002MA  ... S0103_C02_102MA S0103_C02_103E  \
0            0.2            None  ...            None           64.3   

  S0103_C02_103EA S0103_C02_103M S0103_C02_103MA S0103_C02_104E  \
0            None           10.3            None            962   

  S0103_C02_104EA S0103_C02_104M S0103_C02_104MA           ucgid  
0            None             42            None  0500000US08045  

[1 rows x 835 columns]


In [2]:
# Selecting Columns for at risk populations
garfieldSelection = garfield[
    [
        "GEO_ID",
        "NAME",
        "S0103_C01_001E",
        "S0103_C02_001E",
        "S0103_C01_007E",
        "S0103_C01_008E",
        "S0103_C01_009E",
        "S0103_C01_010E",
        "S0103_C01_011E",
        "S0103_C01_012E",
        "S0103_C01_013E",
        "S0103_C01_014E",
        "S0103_C01_044E",
        "S0103_C01_085E",
        "S0103_C01_086E",
    ]
]
garfieldSelection

,GEO_ID,NAME,S0103_C01_001E,S0103_C02_001E,S0103_C01_007E,S0103_C01_008E,S0103_C01_009E,S0103_C01_010E,S0103_C01_011E,S0103_C01_012E,S0103_C01_013E,S0103_C01_014E,S0103_C01_044E,S0103_C01_085E,S0103_C01_086E
0,0500000US08045,"Garfield County, Colorado",61683,8419,0.7,0.4,0.9,0.1,10.0,11.3,29.5,65.4,9.1,8.1,6.4


In [3]:
# Renaming Columns based on column metadata from Census Bureau
garfieldSelection = garfieldSelection.rename(
    {
        "S0103_C01_001E": "Tot_POP",
        "S0103_C01_007E": "Black_PCT",
        "S0103_C01_008E": "NativeAM_PCT",
        "S0103_C01_009E": "Asian_PCT",
        "S0103_C01_010E": "PacificIsl_PCT",
        "S0103_C01_011E": "OtherRace_PCT",
        "S0103_C01_012E": "TwoPlusRace_PCT",
        "S0103_C01_013E": "Hispanic_PCT",
        "S0103_C01_014E": "NotHispanic_PCT",
        "S0103_C01_044E": "WithDisability_PCT",
        "S0103_C01_085E": "PovtBelow100_PCT",
        "S0103_C01_086E": "Povt100to149_PCT",
        "S0103_C02_001E": "65Plus_POP",
    },
    axis="columns",
)
garfieldSelection

,GEO_ID,NAME,Tot_POP,65Plus_POP,Black_PCT,NativeAM_PCT,Asian_PCT,PacificIsl_PCT,OtherRace_PCT,TwoPlusRace_PCT,Hispanic_PCT,NotHispanic_PCT,WithDisability_PCT,PovtBelow100_PCT,Povt100to149_PCT
0,0500000US08045,"Garfield County, Colorado",61683,8419,0.7,0.4,0.9,0.1,10.0,11.3,29.5,65.4,9.1,8.1,6.4
